In [2]:
# Use gregory method to estimate radiative forcing of 4xCO2 at TOA and surface
# By: Ty Janoski
# updated: 05.11.21

In [3]:
# import statements
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats as stats

%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [4]:
# create function for reading in cesm-LE output
# note: each ensemble member starts on a different year
# please keep this in mind when combining datasets
def read_in(exp,mon,ens,var):
    """
    Use xarray to read in a netCDF file.

    Keyword arguments:
    exp -- CO2 scenario
    mon -- starting month in which CO2 is altered
    ens -- ensemble number
    var -- model output variable
    """
    filein = '/dx02/janoski/cesm-LE/output/b40.1850.cam5-lens.'+exp+'.'+str(
        f"{mon:02d}")+'.'+str(f"{ens:02d}")+'.h1_'+var+'.nc'
    return(xr.open_dataset(filein,chunks=None,use_cftime=True))

In [5]:
# create function for taking spatial averages, while weighting for latitude
def spatial_mean(ds_in, lat_bound_s = -91, lat_bound_n = 91):
    """
    Use xarray/numpy to calculate spatial average while weighting for latitude.
    
    Keyword arguments:
    ds_in -- Dataset or DataArray to take the average of
    lat_bound_s -- float, Southern boundary of area to average
    lat_bound_n -- float, Northern boundary of area to average
    """
    zonal = ds_in.mean(dim='lon').sel(lat=slice(lat_bound_s,lat_bound_n))
    weights = np.cos(np.deg2rad(zonal.lat)) / np.sum(np.cos(np.deg2rad(zonal.lat)))
    return((zonal * weights).sum(dim='lat'))

## TOA fluxes, LW and SW together

In [6]:
# make dates to overwrite time axis on data
dates = xr.cftime_range(start="0002-01-01 12:00:00",end="0003-12-31 12:00:00",freq='D',calendar='noleap')

ΔF_as_list = []
ΔF_cs_list = []
ΔTg_list = []

# iterate through all ensemble members
for e in range(1,101,1):
    # ================ALL SKY=============================

    # read in ctrl net SW, LW
    FSNT = read_in('ctrl',1,e,'FSNT').FSNT # positive downward
    FLNT = read_in('ctrl',1,e,'FLNT').FLNT # positive upward

    # net = SW - LW
    net = FSNT - FLNT

    # overwrite time axis
    net['time'] = dates

    # take monthly averages
    net_ctrl = net.resample(time='1MS').mean(dim='time')

    # repeat for 4xCO2
    FSNT = read_in('4xCO2',1,1,'FSNT').FSNT
    FLNT = read_in('4xCO2',1,1,'FLNT').FLNT
    net = FSNT - FLNT
    net['time'] = dates
    net_exp = net.resample(time='1MS').mean(dim='time')

    ΔF_as_list.append(net_exp - net_ctrl)

    # ==================CLEAR SKY========================

    # read in ctrl net SW, LW
    FSNTC = read_in('ctrl',1,1,'FSNTC').FSNTC # positive downward
    FLNTC = read_in('ctrl',1,1,'FLNTC').FLNTC # positive upward

    # net = SW - LW
    net = FSNTC - FLNTC

    # overwrite time axis
    net['time'] = dates

    # take monthly averages
    net_ctrl = net.resample(time='1MS').mean(dim='time')

    # repeat for 4xCO2
    FSNTC = read_in('4xCO2',1,1,'FSNTC').FSNTC
    FLNTC = read_in('4xCO2',1,1,'FLNTC').FLNTC
    net = FSNTC - FLNTC
    net['time'] = dates
    net_exp = net.resample(time='1MS').mean(dim='time')

    ΔF_cs_list.append(net_exp - net_ctrl)

    # load in 2 meter air temp
    ctrl = read_in('ctrl',1,1,'TREFHT').TREFHT
    exp = read_in('4xCO2',1,1,'TREFHT').TREFHT
    ΔT = exp - ctrl
    ΔT['time'] = dates
    ΔTg_list.append(spatial_mean(ΔT).resample(time='1MS').mean(dim='time'))
    
# Concat lists along new ensemble axis
ΔF_as = xr.concat(ΔF_as_list,dim='ens')
ΔF_cs = xr.concat(ΔF_cs_list,dim='ens')
ΔTg = xr.concat(ΔTg_list,dim='ens')

KeyboardInterrupt: 

In [ ]:
# perform linear regression of ΔF vs ΔTg
# we only care about the intercept, so I've adapted the formula for the intercept to operate on these 3D arrays over time axis
x = ΔTg.mean(dim='ens')
y = ΔF_cs.mean(dim='ens')

intercept_cs = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

x = ΔTg.mean(dim='ens')
y = ΔF_as.mean(dim='ens')

intercept_as = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

In [44]:
# rename them, and save them out
intercept_as = intercept_as.rename('dF_as')
intercept_as.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_as.nc')

intercept_cs = intercept_cs.rename('dF_cs')
intercept_cs.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_cs.nc')

## TOA fluxes, LW and SW separate

In [ ]:
# repeat process but separate SW and LW
# make dates to overwrite time axis on data
dates = xr.cftime_range(start="0002-01-01 12:00:00",end="0003-12-31 12:00:00",freq='D',calendar='noleap')

ΔSW_as_list = []
ΔSW_cs_list = []
ΔLW_as_list = []
ΔLW_cs_list = []
ΔTg_list = []

# iterate through all ensemble members
for e in range(1,101,1):
    # ================ALL SKY=============================

    # read in ctrl net SW, LW
    FSNT = read_in('ctrl',1,e,'FSNT').FSNT # positive downward
    FSNT['time'] = dates
    FLNT = -1 * read_in('ctrl',1,e,'FLNT').FLNT # positive upward
    FLNT['time'] = dates

    # take monthly averages
    FSNT_ctrl = FSNT.resample(time='1MS').mean(dim='time')
    FLNT_ctrl = FLNT.resample(time='1MS').mean(dim='time')

    # repeat for 4xCO2
    FSNT = read_in('4xCO2',1,1,'FSNT').FSNT
    FSNT['time'] = dates
    FLNT = -1 * read_in('4xCO2',1,1,'FLNT').FLNT
    FLNT['time'] = dates

    FSNT_exp = FSNT.resample(time='1MS').mean(dim='time')
    FLNT_exp = FLNT.resample(time='1MS').mean(dim='time')

    ΔSW_as_list.append(FSNT_exp - FSNT_ctrl)
    ΔLW_as_list.append(FLNT_exp - FLNT_ctrl)
    # ==================CLEAR SKY========================

    # read in ctrl net SW, LW
    FSNTC = read_in('ctrl',1,e,'FSNTC').FSNTC # positive downward
    FSNTC['time'] = dates
    FLNTC = -1 * read_in('ctrl',1,e,'FLNTC').FLNTC # positive upward
    FLNTC['time'] = dates

    # take monthly averages
    FSNTC_ctrl = FSNTC.resample(time='1MS').mean(dim='time')
    FLNTC_ctrl = FLNTC.resample(time='1MS').mean(dim='time')

    # repeat for 4xCO2
    FSNTC = read_in('4xCO2',1,1,'FSNTC').FSNTC
    FSNTC['time'] = dates
    FLNTC = -1 * read_in('4xCO2',1,1,'FLNTC').FLNTC
    FLNTC['time'] = dates

    FSNTC_exp = FSNTC.resample(time='1MS').mean(dim='time')
    FLNTC_exp = FLNTC.resample(time='1MS').mean(dim='time')

    ΔSW_cs_list.append(FSNTC_exp - FSNTC_ctrl)
    ΔLW_cs_list.append(FLNTC_exp - FLNTC_ctrl)

    # load in 2 meter air temp
    ctrl = read_in('ctrl',1,1,'TREFHT').TREFHT
    exp = read_in('4xCO2',1,1,'TREFHT').TREFHT
    ΔT = exp - ctrl
    ΔT['time'] = dates
    ΔTg_list.append(spatial_mean(ΔT).resample(time='1MS').mean(dim='time'))
    
# Concat lists along new ensemble axis
ΔSW_as = xr.concat(ΔSW_as_list,dim='ens')
ΔSW_cs = xr.concat(ΔSW_cs_list,dim='ens')
ΔLW_as = xr.concat(ΔLW_as_list,dim='ens')
ΔLW_cs = xr.concat(ΔLW_cs_list,dim='ens')
ΔTg = xr.concat(ΔTg_list,dim='ens')

In [ ]:
# perform linear regression of ΔF vs ΔTg
# we only care about the intercept, so I've adapted the formula for the intercept to operate on these 3D arrays over time axis
x = ΔTg.mean(dim='ens')
y = ΔSW_as.mean(dim='ens')

intercept_SW_as = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

x = ΔTg.mean(dim='ens')
y = ΔSW_cs.mean(dim='ens')

intercept_SW_cs = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

x = ΔTg.mean(dim='ens')
y = ΔLW_as.mean(dim='ens')

intercept_LW_as = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

x = ΔTg.mean(dim='ens')
y = ΔLW_cs.mean(dim='ens')

intercept_LW_cs = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

In [26]:
# rename them, and save them out
intercept_SW_as = intercept_SW_as.rename('dFSNT')
intercept_SW_as.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_SW_as.nc')

intercept_LW_as = intercept_LW_as.rename('dFLNT')
intercept_LW_as.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_LW_as.nc')

intercept_SW_cs = intercept_SW_cs.rename('dFSNTC')
intercept_SW_cs.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_SW_cs.nc')

intercept_LW_cs = intercept_LW_cs.rename('dFLNTC')
intercept_LW_cs.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_LW_cs.nc')




## Surface fluxes, LW and SW separate

In [32]:
# create function for reading in cesm-LE output
# note: each ensemble member starts on a different year
# please keep this in mind when combining datasets
def read_in(exp,mon,ens,var):
    """
    Use xarray to read in a netCDF file.

    Keyword arguments:
    exp -- CO2 scenario
    mon -- starting month in which CO2 is altered
    ens -- ensemble number
    var -- model output variable
    """
    filein = '/dx05/janoski/d10/Arctic_Research/cesm-LE/output/b40.1850.cam5-lens.'+exp+'.'+str(
        f"{mon:02d}")+'.'+str(f"{ens:02d}")+'.h1_'+var+'.nc'
    return(xr.open_dataset(filein,chunks=None))

In [33]:
# let's now do the surface fluxes and see if we get something reasonable
# make dates to overwrite time axis on data
dates = xr.cftime_range(start="0002-01-01 12:00:00",end="0003-12-31 12:00:00",freq='D',calendar='noleap')

ΔSW_as_list = []
ΔSW_cs_list = []
ΔLW_as_list = []
ΔLW_cs_list = []
ΔTg_list = []

# iterate through all ensemble members
for e in range(1,101,1):
    # ================ALL SKY=============================

    # read in ctrl net SW, LW
    FSNS = read_in('ctrl',1,e,'FSNS').FSNS # positive downward
    FSNS['time'] = dates
    FLNS = -1 * read_in('ctrl',1,e,'FLNS').FLNS # positive upward
    FLNS['time'] = dates

    # take monthly averages
    FSNS_ctrl = FSNS.resample(time='1MS').mean(dim='time')
    FLNS_ctrl = FLNS.resample(time='1MS').mean(dim='time')

    # repeat for 4xCO2
    FSNS = read_in('4xCO2',1,1,'FSNS').FSNS
    FSNS['time'] = dates
    FLNS = -1 * read_in('4xCO2',1,1,'FLNS').FLNS
    FLNS['time'] = dates

    FSNS_exp = FSNS.resample(time='1MS').mean(dim='time')
    FLNS_exp = FLNS.resample(time='1MS').mean(dim='time')

    ΔSW_as_list.append(FSNS_exp - FSNS_ctrl)
    ΔLW_as_list.append(FLNS_exp - FLNS_ctrl)
    # ==================CLEAR SKY========================

    # read in ctrl net SW, LW
    FSNSC = read_in('ctrl',1,e,'FSNSC').FSNSC # positive downward
    FSNSC['time'] = dates
    FLNSC = -1 * read_in('ctrl',1,e,'FLNSC').FLNSC # positive upward
    FLNSC['time'] = dates

    # take monthly averages
    FSNSC_ctrl = FSNSC.resample(time='1MS').mean(dim='time')
    FLNSC_ctrl = FLNSC.resample(time='1MS').mean(dim='time')

    # repeat for 4xCO2
    FSNSC = read_in('4xCO2',1,1,'FSNSC').FSNSC
    FSNSC['time'] = dates
    FLNSC = -1 * read_in('4xCO2',1,1,'FLNSC').FLNSC
    FLNSC['time'] = dates

    FSNSC_exp = FSNSC.resample(time='1MS').mean(dim='time')
    FLNSC_exp = FLNSC.resample(time='1MS').mean(dim='time')

    ΔSW_cs_list.append(FSNSC_exp - FSNSC_ctrl)
    ΔLW_cs_list.append(FLNSC_exp - FLNSC_ctrl)

    # load in 2 meter air temp
    ctrl = read_in('ctrl',1,1,'TREFHT').TREFHT
    exp = read_in('4xCO2',1,1,'TREFHT').TREFHT
    ΔT = exp - ctrl
    ΔT['time'] = dates
    ΔTg_list.append(spatial_mean(ΔT).resample(time='1MS').mean(dim='time'))
    
# Concat lists along new ensemble axis
ΔSW_as = xr.concat(ΔSW_as_list,dim='ens')
ΔSW_cs = xr.concat(ΔSW_cs_list,dim='ens')
ΔLW_as = xr.concat(ΔLW_as_list,dim='ens')
ΔLW_cs = xr.concat(ΔLW_cs_list,dim='ens')
ΔTg = xr.concat(ΔTg_list,dim='ens')

In [34]:
# perform linear regression of ΔF vs ΔTg
# we only care about the intercept, so I've adapted thespatial_meanula for the intercept to operate on these 3D arrays over time axis
x = ΔTg.mean(dim='ens')
y = ΔSW_as.mean(dim='ens')

intercept_SW_as = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

x = ΔTg.mean(dim='ens')
y = ΔSW_cs.mean(dim='ens')

intercept_SW_cs = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

x = ΔTg.mean(dim='ens')
y = ΔLW_as.mean(dim='ens')

intercept_LW_as = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

x = ΔTg.mean(dim='ens')
y = ΔLW_cs.mean(dim='ens')

intercept_LW_cs = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

In [35]:
print(float(spatial_mean(intercept_LW_as,lat_bound_s=70)))
print(float(spatial_mean(intercept_LW_as)))

print(float(spatial_mean(intercept_LW_cs,lat_bound_s=70)))
print(float(spatial_mean(intercept_LW_cs)))

print(float(spatial_mean(intercept_SW_as,lat_bound_s=70)))
print(float(spatial_mean(intercept_SW_as)))

print(float(spatial_mean(intercept_SW_cs,lat_bound_s=70)))
print(float(spatial_mean(intercept_SW_cs)))

1.3089496163667695
2.1314395847253182
7.351812299743137
4.213401415137091
-0.9218521455541804
0.7312332428733168
-2.72555193258634
-1.0035375248479759


In [27]:
# rename them, and save them out
intercept_SW_as = intercept_SW_as.rename('dFSNS')
intercept_SW_as.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_SW_as_sfc.nc')

intercept_LW_as = intercept_LW_as.rename('dFLNS')
intercept_LW_as.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_LW_as_sfc.nc')

intercept_SW_cs = intercept_SW_cs.rename('dFSNSC')
intercept_SW_cs.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_SW_cs_sfc.nc')

intercept_LW_cs = intercept_LW_cs.rename('dFLNSC')
intercept_LW_cs.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_LW_cs_sfc.nc')




## Sensible and latent heat fluxes

In [29]:
# let's now do the surface fluxes and see if we get something reasonable
# make dates to overwrite time axis on data
dates = xr.cftime_range(start="0002-01-01 12:00:00",end="0003-12-31 12:00:00",freq='D',calendar='noleap')

SHFLX_list = []
LHFLX_list = []
ΔTg_list = []

# iterate through all ensemble members
for e in range(1,101,1):
    # ================ALL SKY=============================

    # read in ctrl net SW, LW
    SHFLX = -1 *read_in('ctrl',1,e,'SHFLX').SHFLX # positive downward
    SHFLX['time'] = dates
    LHFLX = -1 * read_in('ctrl',1,e,'LHFLX').LHFLX # positive downward
    LHFLX['time'] = dates

    # take monthly averages
    SHFLX_ctrl = SHFLX.resample(time='1MS').mean(dim='time')
    LHFLX_ctrl = LHFLX.resample(time='1MS').mean(dim='time')

    # repeat for 4xCO2
    SHFLX = -1 * read_in('4xCO2',1,1,'SHFLX').SHFLX
    SHFLX['time'] = dates
    LHFLX = -1 * read_in('4xCO2',1,1,'LHFLX').LHFLX
    LHFLX['time'] = dates

    SHFLX_exp = SHFLX.resample(time='1MS').mean(dim='time')
    LHFLX_exp = LHFLX.resample(time='1MS').mean(dim='time')

    SHFLX_list.append(SHFLX_exp - SHFLX_ctrl)
    LHFLX_list.append(LHFLX_exp - LHFLX_ctrl)

    # load in 2 meter air temp
    ctrl = read_in('ctrl',1,1,'TREFHT').TREFHT
    exp = read_in('4xCO2',1,1,'TREFHT').TREFHT
    ΔT = exp - ctrl
    ΔT['time'] = dates
    ΔTg_list.append(spatial_mean(ΔT).resample(time='1MS').mean(dim='time'))
    
# # Concat lists along new ensemble axis
ΔSHFLX = xr.concat(SHFLX_list,dim='ens')
ΔLHFLX = xr.concat(LHFLX_list,dim='ens')
ΔTg = xr.concat(ΔTg_list,dim='ens')

In [30]:
# perform linear regression of ΔF vs ΔTg
# we only care about the intercept, so I've adapted the formula for the intercept to operate on these 3D arrays over time axis
x = ΔTg.mean(dim='ens')
y = ΔSHFLX.mean(dim='ens')

intercept_SHFLX = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

x = ΔTg.mean(dim='ens')
y = ΔLHFLX.mean(dim='ens')

intercept_LHFLX = ((y.sum(dim='time') * (x**2).sum(dim='time')) - (x.sum(dim='time') * (x*y).sum(dim='time'))) / ((len(y.time) * (x**2).sum(dim='time') - (x.sum(dim='time'))**2))

In [38]:
print(spatial_mean(intercept_SHFLX,lat_bound_s=70))
print(spatial_mean(intercept_SHFLX))

print(spatial_mean(intercept_LHFLX,lat_bound_s=70))
print(spatial_mean(intercept_LHFLX))

<xarray.DataArray ()>
array(0.576985)
<xarray.DataArray ()>
array(-1.13490212)
<xarray.DataArray ()>
array(1.97720263)
<xarray.DataArray ()>
array(4.67601405)


In [50]:
# rename them, and save them out
intercept_SHFLX = intercept_SHFLX.rename('dSHFLX')
intercept_SHFLX.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_SHFLX.nc')

intercept_LHFLX = intercept_LHFLX.rename('dLHFLX')
intercept_LHFLX.to_netcdf('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_LHFLX.nc')



In [47]:
(intercept_LW_cs + intercept_SW_cs).plot()

<Figure size 432x288 with 2 Axes>